<a href="https://colab.research.google.com/github/gnye8/Solubility/blob/main/notebooks/logS_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
X_train = np.load('/content/X_train_standard.npy')
X_test = np.load('/content/X_test_standard.npy')
y_train = np.load('/content/y_train.npy')
y_test = np.load('/content/y_test.npy')

extval = np.load('/content/logS_validation.npy')
extval_df = pd.read_csv('/content/ext_validation_descriptors.csv', index_col=0)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

RF_best = RandomForestRegressor(n_estimators=1600, min_samples_split=2, min_samples_leaf=1,
                                max_features='auto', max_depth=30)

RF_best.fit(X_train, y_train.ravel())

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


In [ ]:
RF_best_preds = RF_best.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

print(f'MAE of Random Forest Regressor with Best Settings:' + str(mean_absolute_error(y_test, RF_best_preds)))
print(f'RMSE of Random Forest Regressor with Best Settings:' + str(mean_squared_error(y_test, RF_best_preds, squared=False)))

Now to use our trained model to predict melting points of our logS external validation dataset.

In [ ]:
extval_mp = RF_best.predict(extval)

In [ ]:
extval_df['RF_best_std_temp_preds'] = extval_mp
extval_df.to_csv('/content/extval_meltingpoints.csv')

Now to build a linear regression model to try to predict solubility (logS) of this dataset.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

X = extval_df[['MolLogP', 'RF_best_std_temp_preds']].to_numpy()
y = extval_df[['logS']].to_numpy()

X_logS_train, X_logS_test, y_logS_train, y_logS_test = train_test_split(X, y, random_state=42)

print(X_logS_train.shape)
print(X_logS_test.shape)
print(y_logS_train.shape)
print(y_logS_test.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler

SS_scaler = StandardScaler()

SS_scaler.fit(X_logS_train)
X_logS_train = SS_scaler.transform(X_logS_train)
X_logS_test = SS_scaler.transform(X_logS_test)

In [ ]:
lin = LinearRegression()

lin.fit(X_logS_train, y_logS_train)
lin.score(X_logS_train, y_logS_train)

In [ ]:
y_logS_preds = lin.predict(X_logS_test)

In [ ]:
print(f'MAE of linear regression model predicting logS:' + str(mean_absolute_error(y_logS_test, y_logS_preds)))
print(f'RMSE of linear regression model predicting logS:' + str(mean_squared_error(y_logS_test, y_logS_preds, squared=False)))